In [2]:
!pip install pulp
from pulp import *

In [3]:
# Creates a list of the scenarios
scenarios = ['MEDIUM', 'HIGH', 'LOW']
crops = ['Corn', 'Sugar Beets','Wheat']
products = ['Corn', 'Sugar Beets HighProfit', 'Sugar Beets LowProfit', 'Wheat']

yields = {'Wheat': 2.5, 
         'Corn': 3,
         'Sugar Beets': 20}

productCropMapping = {'Wheat': 'Wheat', 
         'Corn': 'Corn',
         'Sugar Beets HighProfit': 'Sugar Beets',
         'Sugar Beets LowProfit': 'Sugar Beets'}

maxSale = {'Wheat': -1, 
         'Corn': -1,
         'Sugar Beets HighProfit': 6000,
         'Sugar Beets LowProfit': -1
}

yieldModifier = {'HIGH': 1.2, 
         'MEDIUM': 1,
         'LOW': 0.8}

plantingCost = {'Wheat': 150, 
         'Corn': 230,
         'Sugar Beets': 260}
    
salesPrice = {'Wheat': 170, 
         'Corn': 150,
         'Sugar Beets HighProfit': 36,
         'Sugar Beets LowProfit': 10}
    
purchasePrice =  {'Wheat': 238, 
         'Corn': 210,
         'Sugar Beets': 99999}
    
requirements = {'Wheat': 200, 
         'Corn': 240,
            'Sugar Beets': 0}

# A dictionary of the probability of each scenario
probability = {'HIGH': 1/3, 
         'MEDIUM': 1/3,
         'LOW': 1/3}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Farmer Problem", LpMaximize)

In [4]:
# Create decision variables
plant = LpVariable.dicts("Planting",crops,0) # non-recourse variable
sales = LpVariable.dicts("Period Sales",list(itertools.product(products,scenarios)),0,cat="Integer") # recourse variables
purchase = LpVariable.dicts("Purchase",list(itertools.product(crops,scenarios,)),0,cat="Integer") # recourse variables

# Create objective function
prob += lpSum([probability[i]*lpSum([sales[j,i]*100*salesPrice[j] for j in products]) - probability[i]*lpSum([purchase[j,i]*100*purchasePrice[j] for j in purchasePrice.keys()]) for i in scenarios]) - lpSum([plant[i] * plantingCost[i] for i in crops])

prob += lpSum(plant[i] for i in crops) <= 500
  
for i in scenarios:
    for j in crops:
        # Inventory constraint
        prob += yieldModifier[i]*yields[j]*plant[j]+purchase[j,i]*100 - lpSum(sales[p,i]*100 for p in products if productCropMapping[p]==j) >= requirements[j], "Feed %s %s" %(i,j)
    for j in products:
        if maxSale[j]>=0:
            prob += sales[j,i]*100<=maxSale[j], "Sales Limit %s %s" %(j,i)
        

In [5]:
print(prob)

Farmer Problem:
MAXIMIZE
5000.0*Period_Sales_('Corn',_'HIGH') + 5000.0*Period_Sales_('Corn',_'LOW') + 5000.0*Period_Sales_('Corn',_'MEDIUM') + 1200.0*Period_Sales_('Sugar_Beets_HighProfit',_'HIGH') + 1200.0*Period_Sales_('Sugar_Beets_HighProfit',_'LOW') + 1200.0*Period_Sales_('Sugar_Beets_HighProfit',_'MEDIUM') + 333.3333333333333*Period_Sales_('Sugar_Beets_LowProfit',_'HIGH') + 333.3333333333333*Period_Sales_('Sugar_Beets_LowProfit',_'LOW') + 333.3333333333333*Period_Sales_('Sugar_Beets_LowProfit',_'MEDIUM') + 5666.666666666666*Period_Sales_('Wheat',_'HIGH') + 5666.666666666666*Period_Sales_('Wheat',_'LOW') + 5666.666666666666*Period_Sales_('Wheat',_'MEDIUM') + -230*Planting_Corn + -260*Planting_Sugar_Beets + -150*Planting_Wheat + -7000.0*Purchase_('Corn',_'HIGH') + -7000.0*Purchase_('Corn',_'LOW') + -7000.0*Purchase_('Corn',_'MEDIUM') + -3333300.0*Purchase_('Sugar_Beets',_'HIGH') + -3333300.0*Purchase_('Sugar_Beets',_'LOW') + -3333300.0*Purchase_('Sugar_Beets',_'MEDIUM') + -7933.3333

In [7]:
# Optimize

prob.solve()



# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])
# Print the value of the objective
print("Objective = %f" % value(prob.objective))


# Print the value of the variables at the optimum
for c in crops:
    print("%s = %f" % (plant[c].name, plant[c].varValue))

for j in sales.keys():
    print("%s %s" %(sales[j].name,sales[j].varValue*100))
    
for j in purchase.keys():
    if purchasePrice[j[0]] < 1000:
        print("%s %s" %(purchase[j].name,purchase[j].varValue*100))

    
stochasticObjective = value(prob.objective)

Status = Optimal
Objective = 101166.666667
Planting_Corn = 150.000000
Planting_Sugar_Beets = 250.000000
Planting_Wheat = 100.000000
Period_Sales_('Corn',_'MEDIUM') 200.0
Period_Sales_('Corn',_'HIGH') 300.0
Period_Sales_('Corn',_'LOW') 100.0
Period_Sales_('Sugar_Beets_HighProfit',_'MEDIUM') 5000.0
Period_Sales_('Sugar_Beets_HighProfit',_'HIGH') 6000.0
Period_Sales_('Sugar_Beets_HighProfit',_'LOW') 4000.0
Period_Sales_('Sugar_Beets_LowProfit',_'MEDIUM') 0.0
Period_Sales_('Sugar_Beets_LowProfit',_'HIGH') 0.0
Period_Sales_('Sugar_Beets_LowProfit',_'LOW') 0.0
Period_Sales_('Wheat',_'MEDIUM') 0.0
Period_Sales_('Wheat',_'HIGH') 100.0
Period_Sales_('Wheat',_'LOW') 0.0
Purchase_('Corn',_'MEDIUM') 0.0
Purchase_('Corn',_'HIGH') 0.0
Purchase_('Corn',_'LOW') 0.0
Purchase_('Wheat',_'MEDIUM') 0.0
Purchase_('Wheat',_'HIGH') 0.0
Purchase_('Wheat',_'LOW') 0.0
